In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms

In [19]:
# 从二级制数据读取文件
import torch.utils.data as Data
import os
import gzip
import numpy as np

dataPath = '/home/zuoyuhui/datasets/fashion-mnist'

def load_data(data_folder, data_name, label_name):
    """
        data_folder: 文件目录
        data_name： 数据文件名
        label_name：标签数据文件名
    """
    with gzip.open(os.path.join(data_folder,label_name), 'rb') as lbpath: # rb表示的是读取二进制数据
        y_train = np.frombuffer(lbpath.read(), np.uint8, offset=8)
 
    with gzip.open(os.path.join(data_folder,data_name), 'rb') as imgpath:
        x_train = np.frombuffer(
            imgpath.read(), np.uint8, offset=16).reshape(len(y_train), 28, 28)
    return (x_train, y_train)


class DealDataset(Data.Dataset):
    """
        读取数据、初始化数据
    """
    def __init__(self, folder, data_name, label_name,transform=None):
        (train_set, train_labels) = load_data(folder, data_name, label_name) # 其实也可以直接使用torch.load(),读取之后的结果为torch.Tensor形式
        self.train_set = train_set
        self.train_labels = train_labels
        self.transform = transform
 
    def __getitem__(self, index):
 
        img, target = self.train_set[index], int(self.train_labels[index])
        if self.transform is not None:
            img = self.transform(img)
        return img, target
 
    def __len__(self):
        return len(self.train_set)

In [24]:

# 实例化这个类，然后我们就得到了Dataset类型的数据，记下来就将这个类传给DataLoader，就可以了。
trainDataset = DealDataset(dataPath,
                           "train-images-idx3-ubyte.gz",
                           "train-labels-idx1-ubyte.gz",
                           transform=transforms.Compose([
                               transforms.ToTensor()
                               transforms.Normalizemalize((0.2860,),(0.3530,))
                           ]))
 
testDataset = DealDataset(dataPath, train=False,
                          "t10k-images-idx3-ubyte.gz",
                          "t10k-labels-idx1-ubyte.gz",
                          transform=transforms.Compose([
                               transforms.ToTensor()
                               transforms.Normalizemalize((0.2860,),(0.3530,))
                           ]))




In [25]:
trainDataset[223][0].shape

/root/anaconda3/lib/python3.7/site-packages/torchvision/transforms/functional.py:74: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729006826/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


torch.Size([1, 28, 28])

In [41]:
data = [d[0].data.cpu().numpy() for d in trainDataset]
print(np.mean(data),np.std(data)) # 所有图片像素的均值方差

len(data)

0.2860402 0.3530239


60000

In [31]:
# 10个类别的图像，分别是：t-shirt（T恤），trouser（牛仔裤），pullover（套衫），dress（裙子），coat（外套），sandal（凉鞋），shirt（衬衫），sneaker（运动鞋），bag（包），ankle boot（短靴）。
# 这里trainDataset包含:train_labels, train_set等属性;  数据类型均为ndarray
print(f'trainDataset.train_labels.shape:{trainDataset.train_labels.shape}\n')
print(f'trainDataset.train_set.shape:{trainDataset.train_set.shape}\n')
 
# 这里train_loader包含:batch_size、dataset等属性，数据类型分别为int，DealDataset
# dataset中又包含train_labels, train_set等属性;  数据类型均为ndarray
# print(f'train_loader.batch_size: {train_loader.batch_size}\n')
# print(f'train_loader.dataset.train_labels.shape: {train_loader.dataset.train_labels.shape}\n')
# print(f'train_loader.dataset.train_set.shape: {train_loader.dataset.train_set.shape}\n')


trainDataset.train_labels.shape:(60000,)

trainDataset.train_set.shape:(60000, 28, 28)



In [32]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1) # 28 * 28 -> (28+1-5) 24 * 24
        self.conv2 = nn.Conv2d(20, 50, 5, 1) # 20 * 20
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        # x: 1 * 28 * 28
        x = F.relu(self.conv1(x)) # 20 * 24 * 24
        x = F.max_pool2d(x,2,2) # 12 * 12
        x = F.relu(self.conv2(x)) # 8 * 8
        x = F.max_pool2d(x,2,2) # 4 *4 
        x = x.view(-1, 4*4*50) # reshape (5 * 2 * 10), view(5, 20) -> (5 * 20)
        x = F.relu(self.fc1(x))
        x= self.fc2(x)
        # return x
        return F.log_softmax(x, dim=1) # log probability

In [51]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for idx,(data,target) in enumerate(train_loader):
        data,target = data.to(device), target.to(device)
        
        pred = model(data)
        # NLLLoss 函数输入 input 之前，需要对 input 进行 log_softmax 处理
        loss = F.nll_loss(pred,target) # 对每个标签对应列进行绝对值取平均
        
        #SGD
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if idx % 100 ==0:
            print('Train Epoch:{}, iteration:{} , Loss:{}'.format(epoch,idx,loss.item()))
            
            
def test(model, device, test_loader):
    model.eval()
    
    total_loss=0.
    correct = 0.
    with torch.no_grad():
        for idx,(data,target) in enumerate(test_loader):
            data,target = data.to(device), target.to(device)

            output = model(data) # batch_size*10 找出最大的
            # NLLLoss 函数输入 input 之前，需要对 input 进行 log_softmax 处理
            total_loss += F.nll_loss(output, target, reduction="sum").item()  # 对每个标签对应列进行绝对值取求和
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum().item()

                
    total_loss /= len(test_loader.dataset)
    acc = correct/len(test_loader.dataset) * 100.
    print('Test loss:{},Accuracy:{}'.format(total_loss,acc))

In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 32
# 训练数据和测试数据的装载
train_loader = Data.DataLoader(
    dataset=trainDataset,
    batch_size=batch_size, 
    shuffle=True,
    num_workers=1,
    pin_memory=True, #可让训练速度变快
)
 
test_loader = Data.DataLoader(
    dataset=testDataset,
    batch_size=100,
    shuffle=False,
    num_workers=1,
    pin_memory=True, #可让训练速度变快
)

lr = 0.01
momentum = 0.5
model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=lr,momentum=momentum)

num_epochs = 2
for epoch in range(num_epochs):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    
    
torch.save(model.state_dict() , 'fm_cnn.pt')
    

Train Epoch:0, iteration:0 , Loss:2.294975996017456
Train Epoch:0, iteration:100 , Loss:1.3832156658172607
Train Epoch:0, iteration:200 , Loss:0.8337352275848389
Train Epoch:0, iteration:300 , Loss:0.917647659778595
Train Epoch:0, iteration:400 , Loss:0.5856406092643738
Train Epoch:0, iteration:500 , Loss:0.6871902942657471
Train Epoch:0, iteration:600 , Loss:0.8883571624755859
Train Epoch:0, iteration:700 , Loss:0.6252657771110535
Train Epoch:0, iteration:800 , Loss:0.8801292181015015
Train Epoch:0, iteration:900 , Loss:0.7535462975502014
Train Epoch:0, iteration:1000 , Loss:0.515783965587616
Train Epoch:0, iteration:1100 , Loss:0.6451463103294373
Train Epoch:0, iteration:1200 , Loss:0.6824079751968384
Train Epoch:0, iteration:1300 , Loss:0.4769243597984314
Train Epoch:0, iteration:1400 , Loss:0.5080010294914246
Train Epoch:0, iteration:1500 , Loss:0.7674784660339355
Train Epoch:0, iteration:1600 , Loss:0.9188860654830933
Train Epoch:0, iteration:1700 , Loss:0.4896189272403717
Train E